In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import scipy.io as sio
import sys
sys.path.append('./helpers')
import loadmat as lm
from sklearn import linear_model
from scipy import signal
from sklearn.model_selection import cross_val_score, cross_validate
import os
from sklearn.metrics import make_scorer, confusion_matrix
import glob
from sklearn.metrics import mean_absolute_error
import seaborn as sns
from helpers import preprocess

In [2]:
files = glob.glob('Z:/giocomo/attialex/NP_DATA/*_gain_*.mat')
dataset = lm.loadmat(files[1])
dataset_p = preprocess(dataset,5)


In [4]:
#dataset_p = preprocess(dataset,5)
bl_trials = np.nonzero(np.all([dataset_p['trial_contrast']==100, dataset_p['trial_gain']==1],axis = 0))
trialidx = np.isin(dataset_p['trial_resampled'],bl_trials)
model = linear_model.LinearRegression()
model.fit(dataset_p['spikecount'][trialidx,:],dataset_p['posx_resampled'][trialidx])


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [17]:
files[1]

'Z:/giocomo/attialex/NP_DATA\\AA1_190729_gain_1.mat'

In [7]:
def score_baseline_model(model,test_data):
    ma_errors = []
    m_errors = []
    precision = []
    conf_matrix = []
    for gain in [1, 0.8, 0.7, 0.6, 0.5]:
        

        bl_trials = np.nonzero(np.all([test_data['trial_contrast']==100, test_data['trial_gain']==gain],axis = 0))
        trialidx = np.in1d(test_data['trial_resampled'],bl_trials)
        if trialidx.sum()>0:
            #print(model.score(test_data['spikecount'][trialidx,:],test_data['posx_bin'][trialidx]))
            Yhat = model.predict(test_data['spikecount'][trialidx,:])
            ma_errors.append(mean_absolute_error(Yhat,test_data['posx_resampled'][trialidx]))
            precision.append(model.score(test_data['spikecount'][trialidx,:],test_data['posx_resampled'][trialidx]))
            err = Yhat-test_data['posx_resampled'][trialidx]
            m_errors.append(err.mean())
            #c_matrix=confusion_matrix(test_data['posx_bin'][trialidx],Yhat)
            #conf_matrix.append(c_matrix)
        else: 
            ma_errors.append(np.nan)
            m_errors.append(np.nan)
            precision.append(np.nan)
            #conf_matrix.append(np.nan)
        
        
    return (ma_errors,m_errors,precision)

In [15]:
bl_trials = np.nonzero(np.all([dataset_p['trial_contrast']==100, dataset_p['trial_gain']==0.8],axis = 0))
trialidx = np.isin(dataset_p['trial_resampled'],bl_trials)

In [41]:
idx = anatomy['cluster_parent']=='VISp'
dataset_p['spikecount'].shape
idxG=idx[dataset_p['sp']['cgs']==2]
vis_cells = dataset_p['spikecount'][:,idxG]
idxG.sum()

C:\Users\attialex\AppData\Local\Continuum\anaconda3\envs\panda3d\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


94

In [50]:
region = 'RSP'
group = anatomy['cluster_parent']
idx = [region in ss for ss in group]
idx = np.array(idx)
idx = idx[dataset['sp']['cgs']==2]


C:\Users\attialex\AppData\Local\Continuum\anaconda3\envs\panda3d\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until
